# Intro To Cairo Graphics With Qahirah #


[Cairo](https://www.cairographics.org/) is a powerful 2D graphics API, with many useful capabilities. Qahirah is a Cairo wrapper for Python which tries to take advantage of Python’s language versatility. The aim of Qahirah is to produce a binding that makes Cairo look as though it was written for Python. If you find the [Cairo reference material](https://www.cairographics.org/manual/) a bit overwhelming, hopefully this notebook will shed some light on at least the basics.

In [ ]:
from IPython.display import \
    display_png

import qahirah as qah
from qahirah import \
    CAIRO, \
    Colour, \
    Vector

## Basic Cairo Objects: Surface And Context ##

The Cairo API consists of a number of different object types, represented in Qahirah by corresponding Python classes. The core of them is the _surface_ and the _context_.

A _surface_ (type [`cairo_surface_t`](https://www.cairographics.org/manual/cairo-cairo-surface-t.html) in Cairo, class `Surface` in Qahirah) holds the result of drawing. There are different types of surfaces, perhaps the most common being an [_image surface_](https://www.cairographics.org/manual/cairo-Image-Surfaces.html), represented by the `ImageSurface` type in Qahirah. This holds a two-dimensional array of pixels, of a specified width, height and format. Other surface types allow rendering directly to file formats such as PDF and SVG, but this introduction will concentrate on image surfaces.

While Cairo supports different [formats](https://www.cairographics.org/manual/cairo-Image-Surfaces.html#cairo-format-t) of image surfaces, the most common ones you are likely to encounter are `CAIRO.IMAGE_FORMAT_RGB24` and `CAIRO.IMAGE_FORMAT_ARGB32` (to use their names in Qahirah). The former includes 8 bits in a pixel for each of the red, green and blue components of the colour, while the latter adds 8 bits of _alpha transparency_ , which can be used to produce overlay effects.

In [ ]:
pix = qah.ImageSurface.create \
  (
    dimensions = (256, 256),
    format = CAIRO.FORMAT_ARGB32
  )

figure_dimensions = pix.dimensions

def display() :
    "(re)displays what has been drawn."
    display_png(pix.to_png_bytes(), raw = True)
#end display

A _context_ (type [`cairo_t`](https://www.cairographics.org/manual/cairo-cairo-t.html) in Cairo, class `Context` in Qahirah) holds settings for drawing into a surface. These consist of information such as what font to use for text, what line thickness and dashing to use for drawing lines, and what colour or pattern to use to actually render pixels. This is all basically information that is only needed while drawing is taking place, and can be discarded afterwards.

A context is created by specifying a surface into which it will do its drawing:

In [ ]:
ctx = qah.Context.create(pix)

Pixels in an `ImageSurface` are commonly undefined to begin with; they can all be initialized to a common colour or pattern with the `paint()` method:

In [ ]:
ctx.source_colour = Colour.x11["sea green"]
ctx.paint()

display()

## Basic Geometry: Paths ##

Shapes drawn in Cairo are represented as [_paths_](https://www.cairographics.org/manual/cairo-Paths.html). Cairo itself provides very limited tools for manipulating these, but Qahirah wraps them in its own `Path` class, which offers a lot more.

A path consists of one or more disjoint _segments_ , each of which is defined by a connected series of straight lines and curves. A segment may be _closed_ , in which case the end point of the last line or curve is implicitly joined to the start point of the first line or curve, or it may be _open_ , in which case no such joining is implied.

To actually draw pixels into a surface, the path has to be either _filled_ or _stroked_. _Filling_ a path means drawing the current source colour or pattern into its interior bounds, while _stroking_ means the source colour or pattern is used to draw a line of specified thickness along the path. (Note that filling an open path actually causes it to be treated as a _closed_ path.)

_Coordinates_ of points in Cairo are specified as pairs of floating-point numbers $(x, y)$. In the _default_ coordinate system, $x$ values increase from left to right, while $y$ values increase from top to bottom, with $(x, y) = (0, 0)$ located at the top left of the drawing surface.

When drawing into an `ImageSurface` with the default transformation, these coordinates map directly to pixel positions, rounded to the nearest integer. However, the ability to specify fractional coordinates is actually very useful, when doing _resolution-independent_ drawing. This will become apparent later.

In Cairo, path construction is done by specifying point coordinates as separate _x_ and _y_ arguments. In Qahirah, these are combined into a single `Vector` object. More will be divulged about this later, but for now, just note the need for an extra pair of parentheses around coordinate argument pairs.


In [ ]:
ctx.move_to((100, 100))
ctx.line_to((200, 200))
ctx.source_colour = Colour.x11["goldenrod"]
ctx.line_width = 4
ctx.stroke()

display()

Qahirah allows _method chaining_ of drawing calls in a `Context` object. This means that each such method call ends with `return self`, allowing another method call to be directly appended. This is useful because such calls commonly come in long sequences. Also, you have already seen the setting of the `source_colour` property of a `Context` to allow drawing with a simple single colour; such properties have `set()` method counterparts which can be used in method chaining.


In [ ]:
(ctx
    .set_source_colour(Colour.x11["turquoise"])
    .paint()
    .move_to((100, 100))
    .line_to((200, 200))
    .line_to((150, 100))
    .set_source_colour(Colour.x11["midnight blue"])
    .fill()
)

display()

Path curves in Cairo are defined in terms of the mathematical entities known as _cubic Bézier curves_. If you are interested in the details, there is an accompanying notebook that goes into them. But for now, just note that a curve segment is defined by four _control points_ : the curve starts at one and ends at another, but it passes “near” the two intermediate points, which can be used to adjust its shape in various useful ways.

In [ ]:
(ctx
    .set_source_colour(Colour.x11["peachpuff1"])
    .paint()
    .set_source_colour(Colour.x11["blue violet"])
    .move_to((50, 50))
    .curve_to((50, 150), (150, 50), (200, 200))
    .set_line_width(8)
    .stroke()
)

display()

But while Bézier curves can be used to produce some quite elaborate designs, there are some simple shapes that are used quite frequently, like circles (or arcs of circles) and rectangles. Cairo has special calls just to produce arcs and (axis-aligned) rectangles; Qahirah also provides a convenience wrapper to draw whole circles.

Note that Qahirah has a special `Rect` type for representing rectangle geometry. This object allows for some useful manipulations.

In [ ]:
rect1 = qah.Rect.from_corners((175, 175), (150, 150))
(ctx
    .set_source_colour(Colour.x11["light pink"])
    .paint()
    .set_source_colour(Colour.x11["deep sky blue"])
    .circle(centre = (100, 100), radius = 35)
    .fill()
    .rectangle(rect1)
    .fill()
    .set_source_colour(Colour.x11["skyblue3"])
    .rectangle(rect1 + Vector(50, -50))
    .fill()
)

display()

## Basic Geometry: Text ##

[Text](https://www.cairographics.org/manual/cairo-text.html) is a subject with many intricacies; some of these are detailed in the “Beginning Text” notebook also included in this set. For now, let us just use the “toy” Cairo calls to do some simple text rendering:

In [ ]:
(ctx
    .set_source_colour(Colour.x11["old lace"])
    .paint()
    .select_font_face("serif", CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_WEIGHT_NORMAL)
    .set_font_size(24)
    .set_source_colour(Colour.x11["deepskyblue4"])
    .move_to((50, 100))
    .show_text("Hello, World!")
)

display()

The shapes of text glyphs can also become part of a path:

In [ ]:
(ctx
    .set_source_colour(Colour.x11["darkseagreen3"])
    .paint()
    .select_font_face("sans serif", CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_WEIGHT_NORMAL)
    .set_font_size(48)
    .move_to((15, 100))
    .text_path("Hello,")
    .move_to((15, 150))
    .text_path("World!")
    .set_source_colour(Colour.x11["thistle3"])
    .fill_preserve()
    .set_source_colour(Colour.x11["mediumorchid4"])
    .set_line_width(2)
    .stroke()
)

display()

Qahirah includes full support for Cairo’s functions for using [FreeType fonts](https://www.cairographics.org/manual/cairo-FreeType-Fonts.html) and even [user-defined fonts](https://www.cairographics.org/manual/cairo-User-Fonts.html).

## Basic Rendering: Source Colours And Patterns ##

So far, we have seen use of some predefined colours from the [traditional X11 set](https://en.wikipedia.org/wiki/X11_color_names) to do our drawing. Cairo itself allows specification of arbitrary colours as [_(red, green, blue, alpha)_ tuples](https://www.cairographics.org/manual/cairo-cairo-t.html#cairo-set-source-rgba), where each component is a floating-point value in the range &#91;0,&nbsp;1&#93; (the alpha component can be omitted if it is 1). However, this is not always the most convenient way to define colours. Thus, Qahirah defines a more elaborate `Colour` class which, in addition to _(r, g, b, a)_ components, also allows colours to be specified in terms of _(h, s, v, a)_ , _(h, l, s, a)_ and _(y, i, q, a)_ components, taking advantage of the conversions provided by the standard Python [`colorsys`](https://docs.python.org/3/library/colorsys.html) module. And there is a convenience method for creating pure greyscales. Plus of course the aforementioned X11 colour names.

The `Colour` class also offers methods for manipulation of colours in various useful ways, to create derived colours that relate to a starting colour according to some operation like changing the brightness/lightness, saturation and so on.

In [ ]:
(ctx
    .set_source_colour(Colour.x11["ivory2"])
    .paint()
)

nrsteps = 9
for i in range(nrsteps) :
    ctx.source_colour = Colour.from_hsva((i / nrsteps, 0.5, 0.75))
    ctx.move_to((128, 128))
    ctx.arc \
      (
        centre = (128, 128),
        radius = 96,
        angle1 = i / nrsteps * qah.circle,
        angle2 = (i + 1) / nrsteps * qah.circle,
        negative = False
      )
    ctx.fill()
#end for

display()

In Cairo, a _source colour_ is merely a special case of a more general _source pattern_. Patterns can also be made up of colour gradients, linear or radial.

In [ ]:
linear_gradient = qah.Pattern.create_linear \
  (
    p0 = figure_dimensions,
    p1 = (0, 0),
    colour_stops =
        (
            (0, Colour.x11["darkgoldenrod4"]),
            (1, Colour.x11["lightgoldenrod"]),
        )
  )
radial_gradient = qah.Pattern.create_radial \
  (
    c0 = figure_dimensions / 2.5,
    r0 = 10,
    c1 = figure_dimensions / 2,
    r1 = abs(figure_dimensions / 2),
    colour_stops =
        (
            (0, Colour.x11["burlywood"]),
            (1, Colour.x11["sienna"]),
        )
  )

(ctx
    .set_source(linear_gradient)
    .paint()
    .set_source(radial_gradient)
    .circle(centre = (128, 128), radius = 96)
    .fill()
)

display()

You can even take the result of drawing into one `Surface`, and turn that image into a `Pattern` for rendering into another surface.

In [ ]:
pix2 = qah.ImageSurface.create \
  (
    dimensions = (96, 96),
    format = CAIRO.FORMAT_RGB24
  )
(qah.Context.create(pix2)
    .set_source_colour(Colour.x11["seashell1"])
    .paint()
    .set_source_colour(Colour.x11["bisque4"])
    .set_line_width(8)
    .move_to((4, 4))
    .line_to((92, 92))
    .move_to((92, 4))
    .line_to((4, 92))
    .stroke()
)
ctx.source_colour = Colour.grey(1)
ctx.paint()
pat = qah.Pattern.create_for_surface(pix2)
pat.extend = CAIRO.EXTEND_REPEAT
ctx.source = pat
ctx.circle(centre = (128, 128), radius = 96)
ctx.fill()

display()

Of course, pattern rendering works with text, too:

In [ ]:
(ctx
    .set_source_colour(Colour.x11["brown4"])
    .paint()
    .select_font_face("sans serif", CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_WEIGHT_NORMAL)
    .set_font_size(72)
    .move_to((15, 100))
    .text_path("Hello,")
    .move_to((15, 200))
    .text_path("World!")
    .set_source(pat)
    .fill()
)

display()

## Line Dashing ##

Cairo doesn’t just let you draw simple lines: you can also define a _dash pattern_ to break the line up into a periodic pattern of segments. This is on top of any source pattern that may be used to render the segments of the line.

In [ ]:
ctx.source_colour = Colour.x11["old lace"]
ctx.paint()
ctx.source_colour = Colour.x11["medium violet red"]
ctx.dash = ((16, 8, 8, 16), 0)
ctx.line_width = 16
ctx.move_to((50, 60))
ctx.line_to((200, 180))
ctx.stroke()

display()


In [ ]:
import array

bwpat = \
  (
    b". ..   ."
    b"  ..    "
    b"      .."
    b"   .. .."
    b".. ..   "
    b"..      "
    b"    ..  "
    b".   .. ."
  )

pat = \
  (
    qah.Pattern.create_for_surface
      (
        qah.ImageSurface.create_for_array
          (
            arr = array.array("B", ((0, 255)[b > ord(" ")] for b in bwpat)),
            format = CAIRO.FORMAT_A8,
            dimensions = (8, 8),
            stride = 8
          )
      )
    .set_filter(CAIRO.FILTER_NEAREST) # for that gritty, pixelated look
    .set_extend(CAIRO.EXTEND_REPEAT)
  )

ctx.source_colour = Colour.x11["old lace"]
ctx.paint()
ctx.source = pat
ctx.dash = ((16, 8, 8, 16), 0)
ctx.line_width = 16
ctx.move_to((50, 60))
ctx.line_to((200, 180))
ctx.stroke()
ctx.dash = ((), 0) # to avoid affecting subsequent drawing

display()


## Vectors And Vector Arithmetic ##

As mentioned, Qahirah does not separate _x_ and _y_ coordinates of points, it combines them into a single `Vector` type. This is because I often see nearly identical calculations written out twice, once to compute _x_ and once to compute _y_. Isn’t it much less effort to just write things out once? Consider the case of computing a point at a given distance and direction from an initial point. This can be done with conventional scalar arithmetic as

    pt2_x = pt1_x + distance * math.cos(direction)
    pt2_y = pt1_y + distance * math.sin(direction)

or it could be done more concisely in Qahirah as

    pt2 = pt1 + Vector.from_polar(distance, direction)

There is a separate notebook in this set, on Vector Arithmetic, that goes into this topic in more detail.

## Basic Coordinate Transformations ##

It is quite common for a figure to occur multiple times in a drawing. The instances might be in different locations, but they can also be rotated into different orientations, or scaled to different sizes.

Rather than try to compute the coordinates directly every time you draw the figure, Cairo can do that for you, by applying a _coordinate transformation_ to the drawing context. Then the code that produces that figure can use a simple fixed set of coordinate numbers.

In computer graphics parlance, the figure is defined in its own _object coordinate system_ , and each instance undergoes a separate transformation to the _world coordinate system_.

In conjunction with this, it can be convenient to use the `save()` and `restore()` methods in a `Context`, so that changes to the coordinate system can be restricted to just part of the drawing. Changes to other context settings also get bracketed in this way.

Note that rotation angles are defined such that 0° is oriented along the positive $x$-axis, while 90° points along the positive $y$-axis. In the default coordinate system, this means that positive rotation angles are _clockwise_ , while negative angles go _anticlockwise_. But this can change if you apply a scaling transformation where the sign of one axis is flipped but not the other.

In [ ]:
ctx.source_colour = Colour.x11["antique white"]
ctx.paint()
ctx.source_colour = Colour.x11["royal blue"]

for i in range(6) :
    ctx.save()
    ctx.rotate(i * 15 * qah.deg)
    ctx.rectangle(qah.Rect(200, 10, 25, 50))
    ctx.fill()
    ctx.restore()
#end for

display()

Applying a global scale factor to the entire drawing is a convenient way to produce graphics at different resolutions: for example, all drawing could be done assuming the default Cairo coordinate system with 72 units to the inch; but the underlying `ImageSurface` could be dimensioned according to a pixel density of 300dpi or 360dpi for high-quality printing, and with an appropriate scale factor (and the use of fractional coordinates where appropriate), the main part of the drawing code does not need to be changed to take account of this.

A more general form of such transformations is availble as available in the form of [matrices](https://www.cairographics.org/manual/cairo-cairo-matrix-t.html), implemented in Qahirah as the `Matrix` type. These can be used to apply positioning, rotation and scaling transformations to contexts, text strings, paths and patterns. There is a whole separate notebook, accompanying this one, that gives more details.

## Clipping ##

It is very common in computer graphics to restrict drawing to just part of the picture. For example, you might be drawing an outside scene visible through a window, some objects of which may only be partly visible. Rather than try to work out how to draw just a _part_ of these objects, it is often easier to define the window frame as a _clipping region_ , and simply draw the entire object; the parts that lie outside the clipping region (hidden by the window edge) do not appear, and existing parts of the drawing in that area remain unchanged instead of being overwritten.

In Cairo, this is done by defining a _clipping path_.

In [ ]:
# without clipping

ctx.source_colour = Colour.x11["honeydew"]
ctx.paint()

ctx.source_colour = Colour.x11["slateblue2"]
ctx.rectangle(qah.Rect(32, 64, 192, 32))
ctx.fill()
ctx.source_colour = Colour.x11["yellow4"]
ctx.rectangle(qah.Rect(32, 128, 192, 32))
ctx.fill()

display()

# with clipping

ctx.source_colour = Colour.x11["honeydew"]
ctx.paint()

ctx.circle(centre = (128, 96), radius = 64)
ctx.clip()

ctx.source_colour = Colour.x11["slateblue2"]
ctx.rectangle(qah.Rect(32, 64, 192, 32))
ctx.fill()
ctx.source_colour = Colour.x11["yellow4"]
ctx.rectangle(qah.Rect(32, 128, 192, 32))
ctx.fill()

ctx.reset_clip() # just to avoid affecting subsequent drawing

display()


## Path Objects ##

It should already be clear that paths are a very fundamental concept in the Cairo graphics model. So it is surprising to see that Cairo itself does not provide much in the way of facilities for manipulating them. But never fear, because Qahirah fills that gap.

Paths are built up in Cairo out of `new_sub_path()`, `move_to()`, `line_to()`, `curve_to()` and `close_path()` calls. (Other shapes like arcs, circles and rectangles are ultimately built out of these same primitives.) Qahirah wraps Cairo’s `copy_path()` call to return a pure-Python `Path` object. It also provides means to construct `Path` objects directly, without using Cairo’s calls, and therefore without having to create a drawing `Context`. Of course, such `Path`s can be passed back to Cairo, where it is expecting one of its own `cairo_path_t` objects; Qahirah will perform the necessary conversion.

Qahirah’s `Path` objects consist of a sequence of `Path.Segment` objects, each representing a sub-path. A `Path.Segment` can be constructed in terms of `MoveTo`, `LineTo`, `CurveTo` and `Close` analogues to Cairo’s path-construction calls, and it can also be decomposed into a sequence of such calls—these elements have become a very traditional way of thinking about paths, dating back to the old PostScript printing/graphics language.

But the underlying representation of a `Path.Segment` is actually in terms of a sequence of `Path.Point` objects: each contains a `Vector` representing its coordinates, together with an _off-curve_ flag: a value of `False` for this flag indicates that the path passes through this point, while a value of `True` indicates that it is an intermediate control point for a Bézier curve.

Thus, a succession of two on-curve points defines a straight line; if there is one off-curve point in-between, then this is a _quadratic_ Bézier segment, and if there are two off-curve points in-between the on-curve points, then this is a _cubic_ Bézier segment (more than two off-curve points in succession are not allowed). The segment also has a flag indicating whether it is open or closed.

This representation for paths lends itself to many useful manipulations. For example, it is easy to reverse the direction of a path, and determine the orientation (clockwise or anticlockwise) of a path segment.


In [ ]:
from qahirah import \
    Path

# three different ways to construct the same path

path1 = \
    (qah.Context.create_for_dummy()
        .move_to((128, 16))
        .curve_to((64, 64), (192, 192), (128, 240))
        .copy_path()
    )
print(path1)

path2 = Path.from_elements \
  (
    [
        Path.MoveTo((128, 16)),
        Path.CurveTo((64, 64), (192, 192), (128, 240)),
    ]
  )
print(path2)

path3 = Path \
  (
    [
        Path.Segment
          (
            points =
                [
                    ((128, 16), False),
                    ((64, 64), True),
                    ((192, 192), True),
                    ((128, 240), False),
                ],
            closed = False
          ),
    ]
  )
print(path3)

In [ ]:
ctx.source_colour = Colour.x11["light steel blue"]
ctx.paint()
ctx.line_width = 16
ctx.source_colour = Colour.x11["mistyrose3"]
ctx.append_path(path2)
ctx.stroke()
ctx.source_colour = Colour.x11["mistyrose4"]
ctx.append_path(path3, qah.Matrix.rotate(angle = 45 * qah.deg, centre = (128, 128)))
ctx.stroke()

display()